In [1]:
# Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
plt.style.use('ggplot')

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [2]:
# Cleaning the text sentences so that punctuation marks, stop words & digits are removed  
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y

### Getting data from SQL server

In [3]:
# Load SQL Connector packages.
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance as sci_distance
from sklearn import cluster as sk_cluster

cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=dbsed4432;DATABASE=pera;Trusted_Connection=yes')

cursor = cnxn.cursor()
data = pd.read_sql_query('SELECT * FROM pera.dbo.eem_l1', cnxn)

cursor.close()


### Data prep

In [4]:
data = data.applymap(str)


round((data.isnull().sum()*100)/len(data),2)


def miss_data(df):
    x = ['column_name','missing_data', 'missing_in_percentage']
    missing_data = pd.DataFrame(columns=x)
    columns = df.columns
    for col in columns:
        icolumn_name = col
        imissing_data = df[col].isnull().sum()
        imissing_in_percentage = (df[col].isnull().sum()/df[col].shape[0])*100

        missing_data.loc[len(missing_data)] = [icolumn_name, imissing_data, imissing_in_percentage]
    print(missing_data)
    
    
miss_data(data)

limitPer = len(data) * .80
df1 = data.dropna(thresh=limitPer,axis=1)


df1=df1.fillna("NA")
df1['mergedentity'] = df1["BIC_Name"].str.lower()+ " "+df1["Billing_Name"].str.lower()+ df1["Billing_NPI"].str.lower()+ " " +df1["Billing_NPPES_NPI_Name"].str.lower()+" "+ df1["Billing_TIN"].str.lower()+" "+ df1["Billing_Adr1"].str.lower()+" " +df1["Billing_City"].str.lower()+df1["Billing_State"].str.lower()+" "+df1["Billing_City"].str.lower()+" "+df1["Billing_Zip"].str.lower()+" "+df1["PayTo_Adr1"].str.lower()+" "+df1["PayTo_City"].str.lower()+" "+df1["PayTo_State"].str.lower()+" "+df1["PayTo_Zip"].str.lower()
df1['mergedentity']=df1['mergedentity'].astype(str)

                        column_name missing_data  missing_in_percentage
0                      HealthSystem            0                    0.0
1                            BIC_ID            0                    0.0
2                          BIC_Name            0                    0.0
3                      Billing_Name            0                    0.0
4                       Billing_NPI            0                    0.0
5         Billing_NPPES_Parent_Name            0                    0.0
6            Billing_NPPES_DBA_Name            0                    0.0
7            Billing_NPPES_NPI_Name            0                    0.0
8                       Billing_TIN            0                    0.0
9             Billing_Taxonomy_Code            0                    0.0
10        Billing_Taxonomy_Grouping            0                    0.0
11  Billing_Taxonomy_Classification            0                    0.0
12  Billing_Taxonomy_Specialization            0                

### Model Training

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df1, test_size=0)


#First column
cols = list(train.columns)
cols = [cols[-1]] + cols[:-1]
train = train[cols]

idea=train.iloc[:,0:1]
corpus=[]
for index,row in idea.iterrows():
    corpus.append(row['mergedentity'])
    
    
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

#vectorizer.get_feature_names()

#print(X.toarray())     

from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)


y_train=train["HealthSystem"]


# Clustering the document with KNN classifier
modelknn = KNeighborsClassifier(n_neighbors=5)
modelknn.fit(X,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

### Model prediction

In [8]:
#First column
cols = list(df1.columns)
cols = [cols[-1]] + cols[:-1]
df1 = df1[cols]


idea=df1.iloc[:,0:1] #Selecting the first column of mergedentity

corpus=[]
for index,row in idea.iterrows():
    corpus.append(row['mergedentity'])
    
Test2 = vectorizer.transform(corpus)

predicted_labels_knn = modelknn.predict(Test2)

idea={'Mergeidentity':corpus, 'Cluster':predicted_labels_knn} #Creating dict having doc with the corresponding cluster number.
frame=pd.DataFrame(idea,index=[predicted_labels_knn], columns=['Idea','Cluster']) # Converting it into a dataframe.

print("\n")
print(frame) #Print the doc with the labeled cluster number.
print("\n")
print(frame['Cluster'].value_counts()) #Print the counts of doc belonging to each cluster.



          Idea    Cluster
Ascension  NaN  Ascension
Ascension  NaN  Ascension
Ascension  NaN  Ascension
Ascension  NaN  Ascension
Ascension  NaN  Ascension
...        ...        ...
TriHealth  NaN  TriHealth
TriHealth  NaN  TriHealth
TriHealth  NaN  TriHealth
TriHealth  NaN  TriHealth
TriHealth  NaN  TriHealth

[4880 rows x 2 columns]


Ascension    2358
Sutter       1122
Tenet         660
NYU           347
Banner        191
Quest         181
Genesis        11
TriHealth      10
Name: Cluster, dtype: int64


In [9]:
df1['Cluster'] = predicted_labels_knn
df1.to_excel (r'C:\Users\rbalani1\Desktop\EEM\IntegratedModel.xlsx', index = False, header=True)

### Upload csv to SQL Server

In [10]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd

cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=dbsed4432;DATABASE=pera;Trusted_Connection=yes')

cursor = cnxn.cursor()

df = pd.read_excel("C:/Users/rbalani1/Desktop/EEM/IntegratedModel.xlsx")
df = df.applymap(str)

for index, row in df.iterrows():
     cursor.execute("INSERT INTO pera.dbo.eem_l1_prediction ([Cluster],[HealthSystem],[BIC_ID],[BIC_Name],[Billing_Name],[Billing_NPI],[Billing_NPPES_NPI_Name],[Billing_TIN],[Billing_Adr1],[Billing_City],[Billing_State],[Billing_Zip],[PayTo_Adr1],[PayTo_City],[PayTo_State],[PayTo_Zip],[Mergedentity]) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row['Cluster'], row['HealthSystem'], row['BIC_ID'], row['BIC_Name'], row['Billing_Name'], row['Billing_NPI'], row['Billing_NPPES_NPI_Name'], row['Billing_TIN'], row['Billing_Adr1'], row['Billing_City'], row['Billing_State'],row['Billing_Zip'], row['PayTo_Adr1'], row['PayTo_City'], row['PayTo_State'], row['PayTo_Zip'], row['mergedentity'])


cnxn.commit()
cursor.close()

### Matched data

In [216]:
df1['Cluster'].isin(df1['HealthSystem']).value_counts()

True    4880
Name: Cluster, dtype: int64